# Tasca S10.01. Analítica de dades en Python

Proyecto de limpieza y análisis de datos en Python usando Pandas.

In [1]:
import pandas as pd
import re
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import os
import jinja2

## Nivell 1

### 1. 
* Importa com un DataFrame l'arxiu sprint10.xlsx. Assegura't que el fitxer s'importa correctament, amb els noms de columnes que li corresponen, sense manipular l'arxiu original.

* Ordena el DataFrame pel país d'origen. En cas d'empat, ordena pel nom de la ciutat.

* Mostra les primeres 10 files.

* Addicionalment, fes un print on comprovi que el DNI només té valors únics.

In [2]:
# importamos el DataFrame
df = pd.read_excel('sprint10.xlsx', header=3)
# ordenamos por país y ciudad
df = df.sort_values(by=["País d'origen","Ciutat"]).reset_index()
# eliminamos columnas de indices
del df['Unnamed: 0'],df['index']
# verificamos que DNI son únicos. comparamos el numero de valores únicos de DNI con el numero de filas 
print(f"Todos los DNI son únicos?: {len(df['DNI'].value_counts())==df.shape[0]}")
# mostramos las primeras 10 filas
df.head(10)

Todos los DNI son únicos?: True


,Nom,Cognoms,DNI,País d'origen,Ciutat,Dia de Naixement,Mes de Naixement,Any de Naixement,Gènere,Salari mensual,Fills,No Fills,Grup Professional
0,Mia,Schneider Fischer,28973553Z,Alemanya,Berlín,22,10,1976,A,951 €,NaN,1.0,Grup A
1,Laura,Schneider Fischer,37399141L,Alemanya,Berlín,2,2,1958,D,1.769 €,1.0,NaN,Grup B
2,Lea,Schneider Schneider,37368317L,Alemanya,Berlín,23,10,2005,D,2.013 €,NaN,1.0,Grup B
3,Mia,Fischer,21390098Z,Alemanya,Berlín,11,8,1950,D,1.557 €,1.0,NaN,Grup B
4,Jonas,Schneider,44060014R,Alemanya,Berlín,22,11,1985,H,2.754 €,1.0,NaN,Grup D
5,Lea,Fischer,14773153R,Alemanya,Berlín,9,9,1986,D,1.370 €,1.0,NaN,Grup A
6,Lea,Müller,23266650S,Alemanya,Hamburg,14,4,2003,D,1.314 €,NaN,1.0,Grup A
7,Anna,Müller,83274277X,Alemanya,Hamburg,1,1,1987,D,2.464 €,NaN,1.0,Grup C
8,Laura,Schmidt Müller,60161784X,Alemanya,Hamburg,15,6,1987,NC,2.035 €,1.0,NaN,Grup C
9,Lukas,Müller Fischer,60982309S,Alemanya,Hamburg,28,3,1971,H,2.042 €,NaN,1.0,Grup B


### 2.
* Crea una columna que sigui el nom complet.

* Crea una columna si la persona és nascuda a Espanya o no.

* Posa el DNI com a índex del DataFrame (noms de files).

* Substitueix el nom de les columnes Dia de Naixement, Mes de Naixement i Any de Naixement per Dia, Mes i Any.

* Substitueix H per Home, D per Dona, A per Altres i NC per una dada faltant (nan/null/na).

Mostra tots els canvis que has realitzat en una sola taula.

In [3]:
# añadimos la columna del nombre completo
df['Nom complet'] = df.apply(lambda row: f"{row['Nom']} {row['Cognoms']}", axis=1)
# añadimos la columna de nacimiento en España
df.loc[df["País d'origen"] == 'Espanya','Naixement Espanya'] = True
df["Naixement Espanya"] = (df["Naixement Espanya"].fillna(False).infer_objects(copy=False))
# ponemos DNI como indice
if "DNI" in df.columns:
    df.index = df["DNI"]
    del df["DNI"]
# cambiamos de nombre a algunas columnas
df = df.rename(columns= {'Dia de Naixement': 'Dia','Mes de Naixement': 'Mes','Any de Naixement': 'Any'})
# sustituimos valores
df = df.replace({'H': 'Home', 'D': 'Dona', 'A': 'Altres', 'NC': pd.NaT})
df.head()

C:\Users\arman\AppData\Local\Temp\ipykernel_1916\977987830.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Naixement Espanya"] = (df["Naixement Espanya"].fillna(False).infer_objects(copy=False))


,Nom,Cognoms,País d'origen,Ciutat,Dia,Mes,Any,Gènere,Salari mensual,Fills,No Fills,Grup Professional,Nom complet,Naixement Espanya
DNI,,,,,,,,,,,,,,
28973553Z,Mia,Schneider Fischer,Alemanya,Berlín,22,10,1976,Altres,951 €,NaN,1.0,Grup A,Mia Schneider Fischer,False
37399141L,Laura,Schneider Fischer,Alemanya,Berlín,2,2,1958,Dona,1.769 €,1.0,NaN,Grup B,Laura Schneider Fischer,False
37368317L,Lea,Schneider Schneider,Alemanya,Berlín,23,10,2005,Dona,2.013 €,NaN,1.0,Grup B,Lea Schneider Schneider,False
21390098Z,Mia,Fischer,Alemanya,Berlín,11,8,1950,Dona,1.557 €,1.0,NaN,Grup B,Mia Fischer,False
44060014R,Jonas,Schneider,Alemanya,Berlín,22,11,1985,Home,2.754 €,1.0,NaN,Grup D,Jonas Schneider,False


### 3.
* Junta les columnes Fills i No Fills en una sola columna, utilitzant el mètode .apply() i definint una funció que resolgui el problema. La columna nova ha de dir-se "Fills" i prendre els valors "Sí" o "No".

In [4]:
def columna_fills(col1,col2):
    if col1 == 1:
        value = 'Sí'
    elif col2 == 1:
        value = 'No'
    else:
        value = pd.NaT
    return value

if ('Fills' in df.columns) and ('No Fills' in df.columns):
    df['Fills'] = df.apply(lambda row: columna_fills(row['Fills'], row['No Fills']), axis=1)
    del df['No Fills']
# print(df['Fills'].unique())
df.head()

,Nom,Cognoms,País d'origen,Ciutat,Dia,Mes,Any,Gènere,Salari mensual,Fills,Grup Professional,Nom complet,Naixement Espanya
DNI,,,,,,,,,,,,,
28973553Z,Mia,Schneider Fischer,Alemanya,Berlín,22,10,1976,Altres,951 €,No,Grup A,Mia Schneider Fischer,False
37399141L,Laura,Schneider Fischer,Alemanya,Berlín,2,2,1958,Dona,1.769 €,Sí,Grup B,Laura Schneider Fischer,False
37368317L,Lea,Schneider Schneider,Alemanya,Berlín,23,10,2005,Dona,2.013 €,No,Grup B,Lea Schneider Schneider,False
21390098Z,Mia,Fischer,Alemanya,Berlín,11,8,1950,Dona,1.557 €,Sí,Grup B,Mia Fischer,False
44060014R,Jonas,Schneider,Alemanya,Berlín,22,11,1985,Home,2.754 €,Sí,Grup D,Jonas Schneider,False


### 4.
* Crea una taula resum que permeti veure el sou mig, medià, mínim i màxim per Gènere.
* Ordena la taula en funció del sou mig.

In [5]:
# convertimos la columna Salari mensual a float
if df['Salari mensual'].dtype != float:
    df['Salari mensual'] = [float(re.sub(r'[^\w\s]', '', salario)) for salario in df['Salari mensual']]
# creamos la tabla y la ordenamos por sueldo medio
df.groupby('Gènere')['Salari mensual'].aggregate(['mean','median','min','max']).sort_values('mean').round(2)

,mean,median,min,max
Gènere,,,,
Dona,1469.44,1361.5,665.0,3021.0
Altres,1626.59,1545.0,703.0,3175.0
Home,1643.25,1531.0,737.0,3356.0


### 5.
* Crea una taula resum amb el salari mig per gènere (files) i país d'origen (columnes).
* Afegeix-hi les mitjanes als marges de la taula.
* (EXTRA): Aplica format condicional a la taula per veure en un color més intens els valors més elevats

In [6]:
# creamos la tabla
table = pd.crosstab(index=df['Gènere'], columns=df["País d\'origen"], values=df["Salari mensual"], aggfunc='mean', 
                    margins=True, margins_name='Mitjana total').round(2)

# formato condicional
table.style.format("{:.2f}").background_gradient()
# sns.heatmap(table, annot=True, fmt=".0f")
# table

País d'origen,Alemanya,Argentina,Colòmbia,Espanya,França,Itàlia,Marroc,Mèxic,Portugal,Regne Unit,Mitjana total
Gènere,,,,,,,,,,,
Altres,951.00,1141.00,1030.00,1706.18,nan,1423.00,1365.00,1372.00,1765.00,1921.00,1626.59
Dona,1804.31,1291.80,1497.75,1460.16,1566.47,1247.18,1405.21,1517.80,1488.55,1489.46,1469.44
Home,2067.43,1583.29,1554.67,1682.11,1389.25,1672.88,1531.00,1625.00,1497.00,1162.56,1643.25
Mitjana total,1851.38,1463.39,1495.54,1581.21,1462.73,1425.95,1447.33,1558.42,1523.33,1423.56,1560.99


### 6.
* Crea una columna nova que sigui la data de naixament en format Datetime a partir de les columnes dia, mes i any. Utilitzant aquesta columna crea una funció que donada una data, et calculi l'edat actual a dia d'avui.
* Utilitza la funció que acabes de crear per generar una columna nova al DataFrame amb l'edat actual.

In [7]:
def edad_actual(fecha, nacimiento):
    '''
    Función que calcula la edad de un usuario respecto a una fecha, 
    que debe tener el formato "%Y-%m-%d" o ser un datetime.
    '''
    if isinstance(fecha, str):
        try:
            fecha = datetime.datetime.strptime(fecha, "%Y-%m-%d")
        except:
            print('Hay un error en el formato de la fecha.')
    
    edad = fecha - nacimiento
    if edad < datetime.timedelta():
        print('El usuario aún no había nacido')
        return
    
    return round(edad.total_seconds() / (365.25 * 24 * 60 * 60))

# creamos la columna Data Naixement
df['Data Naixement'] = [datetime.datetime(row['Any'],row['Mes'],row['Dia'],0,0,0) for _, row in df.iterrows()]
# creamos la columna Edat
df['Edat'] = [edad_actual(datetime.datetime.now(),nacimiento) for nacimiento in df['Data Naixement']]
df.head()


,Nom,Cognoms,País d'origen,Ciutat,Dia,Mes,Any,Gènere,Salari mensual,Fills,Grup Professional,Nom complet,Naixement Espanya,Data Naixement,Edat
DNI,,,,,,,,,,,,,,,
28973553Z,Mia,Schneider Fischer,Alemanya,Berlín,22,10,1976,Altres,951.0,No,Grup A,Mia Schneider Fischer,False,1976-10-22,49
37399141L,Laura,Schneider Fischer,Alemanya,Berlín,2,2,1958,Dona,1769.0,Sí,Grup B,Laura Schneider Fischer,False,1958-02-02,68
37368317L,Lea,Schneider Schneider,Alemanya,Berlín,23,10,2005,Dona,2013.0,No,Grup B,Lea Schneider Schneider,False,2005-10-23,20
21390098Z,Mia,Fischer,Alemanya,Berlín,11,8,1950,Dona,1557.0,Sí,Grup B,Mia Fischer,False,1950-08-11,76
44060014R,Jonas,Schneider,Alemanya,Berlín,22,11,1985,Home,2754.0,Sí,Grup D,Jonas Schneider,False,1985-11-22,40


## Nivell 2

### 1.
* Utilitzant el següent DataFrame, adjunta la columna "Increment" al dataframe del nivell anterior.
* Actualitza la columna salari en funció dels percentatges que s'adjunten. No modifiquis manualment els increments, escriu codi Python per fer les conversions necessàries.

In [8]:
df_increment = pd.DataFrame({"Grup":["Grup A","Grup B","Grup C", "Grup D" ] , "Increment":["5%","3,5%","2%","8%"]})
# juntamos los dos DF
df2 = pd.merge(df, df_increment, left_on="Grup Professional", right_on="Grup", how="left")
del df2['Grup']
df2.index = df.index
# print(df.shape,df2.shape)
# actualizamos la columna de salarios
df2["Salari mensual"] = [ round(row['Salari mensual']*(1+float(row['Increment'][:-1].replace(',','.'))/100),2) for _, row in df2.iterrows()]
# confirmamos que todos son mas grandes
display(all(df['Salari mensual'] < df2['Salari mensual']))
df2.head()

True

,Nom,Cognoms,País d'origen,Ciutat,Dia,Mes,Any,Gènere,Salari mensual,Fills,Grup Professional,Nom complet,Naixement Espanya,Data Naixement,Edat,Increment
DNI,,,,,,,,,,,,,,,,
28973553Z,Mia,Schneider Fischer,Alemanya,Berlín,22,10,1976,Altres,998.55,No,Grup A,Mia Schneider Fischer,False,1976-10-22,49,5%
37399141L,Laura,Schneider Fischer,Alemanya,Berlín,2,2,1958,Dona,1830.91,Sí,Grup B,Laura Schneider Fischer,False,1958-02-02,68,"3,5%"
37368317L,Lea,Schneider Schneider,Alemanya,Berlín,23,10,2005,Dona,2083.45,No,Grup B,Lea Schneider Schneider,False,2005-10-23,20,"3,5%"
21390098Z,Mia,Fischer,Alemanya,Berlín,11,8,1950,Dona,1611.49,Sí,Grup B,Mia Fischer,False,1950-08-11,76,"3,5%"
44060014R,Jonas,Schneider,Alemanya,Berlín,22,11,1985,Home,2974.32,Sí,Grup D,Jonas Schneider,False,1985-11-22,40,8%


### 2.
* Utilitzant un bucle, exporta en 4 fitxers (format .xlsx o .csv) les dades de cada Grup Professional.
* Exporta un 5è DataFrame en format .xlsx o .csv que contingui quants treballadors hi ha per cada Grup Professional, quin és el seu sou mig i quina és la seva edat mediana.

In [9]:
# exportamos 4 ficheros de datos, uno por cada grupo professional
for grupo in df2['Grup Professional'].unique():
    df[df['Grup Professional'] == grupo].to_csv(f'dades_{grupo}.csv', index=True)
# exportamos el siguiente DataFrame
df2.groupby('Grup Professional').aggregate({'Nom': 'count', 'Salari mensual': 'mean', 'Edat': 'median'}).round(2).rename(
    columns={'Nom': 'Nombre treballadors', 'Salari mensual': 'Sou mensual mig', 'Edat': 'Edat mediana'}).to_csv('dades_grups_pro.csv', index=True)

## Nivell 3

### 1.
Crea una funció que prengui un dataframe com a paràmetre d'entrada.
La funció ha de crear (i exportar) un gràfic automàticament per a cada columna del dataframe. Per exemple:
* un histograma/boxplot si la variable és numèrica
* unes barres dels valors més freqüents si és categòrica
* unes barres dels anys més freqüents si la dada està en format data.

La idea és crear una funció que funcioni per qualsevol dataframe, no només amb el que hem treballat fins ara.

In [10]:
def figuras_auto(df, folder = 'figures'):
    '''
    Función que recorre las columnas de un DataFrame y elabora figuras según el tipo.
    Guarda las figuras en un folder. 
        Parameters
        df: DataFrame
        folder: (optional, str) Nombre del folder o path
    '''

    if not os.path.exists(folder):
        os.makedirs(folder)

    for columna in df.columns:
        
        if pd.api.types.is_numeric_dtype(df[columna]) and not pd.api.types.is_bool_dtype(df[columna]):
            print(f'{columna} is number')
            plt.figure()
            df[columna].plot.hist()
            plt.xlabel(columna)
            plt.title(f"{columna} histogram")
            plt.savefig(f"{folder}\\{columna}_figure.jpg")
            plt.close()

        elif pd.api.types.is_bool_dtype(df[columna]):
            print(f"{columna} is boolean")
            plt.figure()
            df[columna].astype(int).plot.hist()
            plt.xlabel(columna)
            plt.title(f"{columna} histogram")
            plt.savefig(f"{folder}\\{columna}_figure.jpg")
            plt.close()

        elif isinstance(df[columna].dtype, pd.CategoricalDtype):
            print(f'{columna} is categorical')
            plt.figure()
            df[columna].value_counts().plot.bar()
            plt.xlabel(columna)
            plt.title(f"{columna} frequency")
            plt.savefig(f"{folder}\\{columna}_figure.jpg")
            plt.close()

        elif pd.api.types.is_datetime64_any_dtype(df[columna]):
            print(f'{columna} is datetime')
            plt.figure()
            df[columna].dt.year.value_counts().sort_index().plot(kind="bar")
            plt.xlabel('Year')
            plt.title(f"{columna} counts per year")
            plt.savefig(f"{folder}\\{columna}_figure.jpg")
            plt.close()

        elif pd.api.types.is_object_dtype(df[columna]):
            print(f'{columna} is object')
            plt.figure()
            df[columna].value_counts().plot.bar()
            plt.xlabel(columna)
            plt.title(f"{columna} frequency")
            plt.savefig(f"{folder}\\{columna}_figure.jpg")
            plt.close()

        else:
            print(f'Data type {df[columna].dtype} is not considered.')
    print('\n')

In [11]:
# datasets exemple: iris, penguins, titanic
for dataset in ['iris', 'penguins', 'titanic']:
    figuras_auto(sns.load_dataset(dataset), folder = dataset)
figuras_auto(df2, folder = 'DataFrame_N2')

sepal_length is number
sepal_width is number
petal_length is number
petal_width is number
species is object


species is object
island is object
bill_length_mm is number
bill_depth_mm is number
flipper_length_mm is number
body_mass_g is number
sex is object


survived is number
pclass is number
sex is object
age is number
sibsp is number
parch is number
fare is number
embarked is object
class is categorical
who is object
adult_male is boolean
deck is categorical
embark_town is object
alive is object
alone is boolean


Nom is object
Cognoms is object
País d'origen is object
Ciutat is object
Dia is number
Mes is number
Any is number
Gènere is object
Salari mensual is number
Fills is object
Grup Professional is object
Nom complet is object
Naixement Espanya is boolean
Data Naixement is datetime
Edat is number
Increment is object




### 2.

Carrega l'arxiu matriu_distancies.xlsx a pandas, de manera que els noms de files i els noms de columnes siguin els de les ciutats. Borra "Las Palmas de Gran Canaria" i "Palma" perquè poguem fer el trajecte en cotxe.

Ens interessa visitar totes les ciutats principals d'Espanya recorrent la mínima distància possible.

Una aproximació senzilla (que no òptima) seria anant sempre a la ciutat més propera que no haguem visitat encara

Fes una funció que donada la matriu de distàncies i la ciutat d'origen, faci una proposta de ruta que sigui el més curta possible que puguis, retornant una llista amb l'ordre de visita. Dóna també la distància total recorreguda.

(EXTRA) Des de quina ciutat la ruta seria més curta amb l'algoritme plantejat

In [12]:
def propuesta_ruta(df,ciudad_origen):
    '''
    Función que elabora una propuesta de ruta.
        Parámetros:
        df: DataFrame, matriz de distancias
        ciudad_origen: str, ciudad inicial de la ruta
        Return:
        ruta: list, nombre de las ciudades de la ruta en orden
        distancia_total: float, distancia total recorrida en km
    '''
    ruta = [ciudad_origen]
    distancia_total = 0
    # lista de ciudades que no estan en la ruta
    ciudades_por_visitar = [ciudad for ciudad in df.columns if ciudad not in ruta]
    # continuar hasta que no haya ciudades por visitar
    while len(ciudades_por_visitar) > 0:
        distancia_minima = df.loc[ciudad_origen,ciudades_por_visitar].min()
        # encontramos la ciudad más cercana por visitar (o ciudades si estan a misma distancia)
        posibles_ciudades = df.columns[df.loc[ciudad_origen] == distancia_minima].tolist()
        # elegimos la ciudad que no hemos visitado de posibles_ciudades
        ciudad_visitada = [ciudad for ciudad in posibles_ciudades if ciudad not in ruta][0]
        # añadir ciudad nueva a la lista ruta
        ruta.append(ciudad_visitada)
        # añadir valor a la distancia total
        distancia_total += distancia_minima
        # cambiamos ciudad de origen
        ciudad_origen = ciudad_visitada
        # cambiamos lista de ciudades por visitar
        ciudades_por_visitar.remove(ciudad_visitada)

    return ruta,distancia_total

In [13]:
# matriz de distancias
df = pd.read_excel('matriu_distancies.xlsx', index_col=0)
del df["Las Palmas de Gran Canaria"], df["Palma"]
df = df.drop(["Las Palmas de Gran Canaria","Palma"])

# buscamos la ciudad con la ruta más corta
ruta_mas_corta = [] #ciudad,ruta,distancia total
for ciudad in df.columns:
    ruta,distancia_total = propuesta_ruta(df,ciudad)
    if (len(ruta_mas_corta)==0)or(ruta_mas_corta[2]>distancia_total):
        ruta_mas_corta = [ciudad,ruta,distancia_total]
print(f"La ciudad con la ruta mas corta es {ruta_mas_corta[0]} con {ruta_mas_corta[2]} km.")
print('ruta:',ruta_mas_corta[1])

La ciudad con la ruta mas corta es Barcelona con 2778.0 km.
ruta: ['Barcelona', 'Hospitalet de Llobregat', 'Zaragoza', 'Valencia', 'Alicante', 'Murcia', 'Córdoba', 'Sevilla', 'Málaga', 'Valladolid', 'Gijón', 'Bilbao', 'Vigo']


## Revisión peer-to-peer
*Revisado por Federico Caruso*

* Considerar que el count solo cuenta valores no nulos. Si hubiera algun valor nulo, no se obtendria el número total de trabajadores correcto.
* Considerar si hace falta dividir el código en más celdas.